In [64]:
%reload_ext autoreload
%autoreload 2
import ast
import pickle
import itertools
from collections import Counter
from tqdm import tqdm
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
import swifter

import sys
sys.path.append('..')
from analysis.generate_cluster_information_file import load, extract_all_information_query, to_df_query
from common.image_processing import pixel_intensity_histogram


from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
import xgboost
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

In [48]:
train_X = pd.read_csv('train_X.csv', dtype='float')
train_y = pd.read_csv('train_y.csv', dtype='float')

na_indices = train_X[train_X.isna().any(axis=1)].index.values
train_X.drop(na_indices, inplace=True)
train_y.drop(na_indices, inplace=True)

print(train_X.shape)
train_X.head(10)

(9600, 16)


,cluster_num,background_threshold,cluster_num_intensities_avg,cluster_num_intensities_max,cluster_num_intensities_min,cluster_num_intensities_std,cluster_peak_intensities_avg,cluster_peak_intensities_max,cluster_peak_intensities_min,cluster_peak_intensities_std,cluster_x_avg,cluster_y_avg,cluster_sizes_avg,cluster_sizes_std,cluster_sizes_max,cluster_sizes_min
0,50.0,0.0,13.844046,1.007527e-311,1.007394e-311,1.007394e-311,14.920000,255.0,1.0,49.370777,513.460000,475.320000,35.580000,46.114679,225.0,1.0
1,104.0,0.0,11.164915,1.007390e-311,1.007390e-311,1.007390e-311,11.375000,255.0,1.0,36.751030,536.201923,473.798077,28.134615,36.295014,179.0,1.0
2,71.0,1.0,12.496173,7.600000e+02,9.100000e+01,1.700000e+02,11.563380,255.0,2.0,34.161890,464.971831,513.746479,34.169014,49.933755,273.0,1.0
3,49.0,2.0,13.074356,7.630000e+02,1.060000e+02,1.810000e+02,22.612245,255.0,3.0,51.612297,417.285714,564.571429,53.632653,107.127941,635.0,1.0
4,169.0,0.0,6.639654,8.010000e+02,1.140000e+02,1.830000e+02,6.621302,255.0,1.0,26.394791,462.224852,509.230769,20.526627,29.501191,161.0,1.0
5,51.0,1.0,17.443819,8.150000e+02,1.170000e+02,1.890000e+02,26.921569,255.0,2.0,64.565772,590.921569,492.352941,47.686275,58.244074,236.0,1.0
6,2216.0,191.0,4.407226,8.330000e+02,1.670000e+02,2.350000e+02,196.498195,254.0,192.0,6.801996,433.569495,468.828520,86.904332,3238.075401,151292.0,1.0
7,66.0,0.0,13.259619,8.360000e+02,1.760000e+02,2.620000e+02,15.045455,255.0,1.0,46.085044,450.257576,427.469697,30.121212,45.954363,263.0,1.0
8,73732.0,9.0,1.713660,8.430000e+02,1.880000e+02,2.910000e+02,17.661884,255.0,10.0,5.843887,498.615499,497.128208,2.412616,2.440176,62.0,1.0
9,35.0,1.0,14.468375,9.050000e+02,2.100000e+02,3.230000e+02,14.171429,255.0,2.0,44.876011,540.771429,514.942857,32.000000,43.131692,224.0,1.0


In [66]:
def train_with_features(X, features, rf=True, svm=True, xgb=True):
    assert len(features) > 0
    assert type(features[0]) == str
    
    if features != 'all':
        X = X[features]
    else:
        X = X
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(train_y)
    
    if rf:
        print("Training Random Forest..")
        randomforest = RandomForestRegressor(max_depth=6, random_state=0)
        score_mi = cross_val_score(randomforest, X_scaled, np.ravel(y_scaled), cv=5, scoring=make_scorer(mean_absolute_error))
        print("Mean: {}, Std Dev: {}".format(score_mi.mean(), np.std(score_mi)))
        
    if svm:
        print("Training SVM..")
        randomforest = SVR(kernel='rbf')
        score_mi = cross_val_score(randomforest, X_scaled, np.ravel(y_scaled), cv=5, scoring=make_scorer(mean_absolute_error))
        print("Mean: {}, Std Dev: {}".format(score_mi.mean(), np.std(score_mi)))
        
    if xgb:
        print("Training XGBoost..")
        model = xgboost.XGBRegressor(objective="reg:squarederror", learning_rate =0.01, booster='gbtree', n_estimators=1000, max_depth=6, gamma=0.5, subsample=0.7, colsample_bytree=1.0, nthread=-1, verbosity=1)
        score_mi = cross_val_score(model, X_scaled, np.ravel(y_scaled), cv=5, scoring=make_scorer(mean_absolute_error))
        print("Mean: {}, Std Dev: {}".format(score_mi.mean(), np.std(score_mi)))

In [42]:
train_with_features(train_X, ['cluster_num'])

Mean: 0.6488447391468939, Std Dev: 0.010385121122722645


### Train with all features

In [53]:
train_with_features(train_X, 'all')

Mean: 0.46932710072849765, Std Dev: 0.007760576881457447


### Train with selected features
We select features according to the analysis in feature_selection.ipynb

In [67]:
filtered_X = train_X.drop(columns=['cluster_sizes_min', 'cluster_num_intensities_min', 'cluster_num_intensities_max', 'cluster_num_intensities_std'])
train_with_features(filtered_X, 'all', rf=False, svm=False)

Training XGBoost..
Mean: 0.41441487988459935, Std Dev: 0.005037303465995161
